In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from jf_wep.donutStamp import DonutStamp
from jf_wep.imageMapper import ImageMapper
from jf_wep.wfAlgorithms.tie import TIEAlgorithm

In [ ]:
# Choose some true Zernikes
zkTrue = np.zeros(19)
zkTrue[0] = -1.1e-6
zkTrue[1] = -0.5e-6
zkTrue[2] = 0.4e-6
zkTrue[3] = -1e-6
zkTrue[4] = -0.2e-6

# Forward model a pair of donuts
mapper = ImageMapper()

intraImg = mapper.mapPupilToImage(
    DonutStamp(
        np.zeros((180, 180)),
        (0, -1),
        "intra",
        "r",
    ),
    zkTrue,
)
intraImg.image *= 100

extraImg = mapper.mapPupilToImage(
    DonutStamp(
        np.zeros((180, 180)),
        (0, -1),
        "extra",
        "r",
    ),
    zkTrue,
)
extraImg.image *= 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5, 2), dpi=180)

ax1.imshow(intraImg.image, origin="lower")
ax1.set(title="Intra")

ax2.imshow(extraImg.image, origin="lower")
ax2.set(title="Extra")

plt.show()

In [ ]:
# Use the TIE to estimate Zernikes
algo = TIEAlgorithm(saveHistory=True, maxIter=30)
zkEst = algo.estimateWf(intraImg, extraImg, instrument=mapper.instrument)

In [ ]:
fig, ax = plt.subplots()

ax.plot(4 + np.arange(len(zkEst)), 1e9 * zkEst, label="Estimated", c="C1")
ax.plot(
    4 + np.arange(len(zkTrue)), 1e9 * zkTrue, label="Truth", c="k", ls="--"
)
ax.legend()

ax.set(xlabel="Zernike Noll index", ylabel="Amplitude (nm)")
plt.show()


In [ ]:
# Plot images from the final iteration of the TIE solver
fig, (ax1, ax2, ax3) = plt.subplots(
    1, 3, figsize=(5, 2.2), dpi=180, constrained_layout=True
)

finalIter = max(algo.history.keys())

ax1.imshow(algo.history[finalIter]["intraComp"], origin="lower")
ax1.set(title="intraComp")

ax2.imshow(algo.history[finalIter]["extraComp"], origin="lower")
ax2.set(title="extraComp")

ax3.imshow(algo.history[finalIter]["dIdz"], origin="lower")
ax3.set(title="dIdz")

fig.suptitle("Images from final iteration of TIE solver")

plt.show()